# GraphX Examples

In [32]:
// Set log level to ERROR (less verbose)
sc.setLogLevel("ERROR")

## Example 1: Research Lab

In [33]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [3]:
// Let's create nodes
val users: RDD[(VertexId, (String, String))] = sc.parallelize(Array(
    (3L, ("rxin", "student")), 
    (7L, ("jgonzal", "postdoc")), 
    (5L, ("franklin", "prof")), 
    (2L, ("istoica", "prof"))))

users.foreach(println)

(3,(rxin,student))
(7,(jgonzal,postdoc))
(5,(franklin,prof))
(2,(istoica,prof))


users: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at <console>:29


In [4]:
// Let's add edges
val relationships: RDD[Edge[String]] = sc.parallelize(Array(
    Edge(3L, 7L, "collab"), 
    Edge(5L, 3L, "advisor"), 
    Edge(2L, 5L, "colleague"), 
    Edge(5L, 7L, "pi")))

relationships.foreach(println)

Edge(3,7,collab)
Edge(5,3,advisor)
Edge(2,5,colleague)
Edge(5,7,pi)


relationships: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at parallelize at <console>:29


In [5]:
// Just in case a node is refered but missing
val defaultUser = ("John Doe", "Missing")

defaultUser: (String, String) = (John Doe,Missing)


In [6]:
// Put all of them together and create a graph
val graph = Graph(users, relationships, defaultUser)

graph: org.apache.spark.graphx.Graph[(String, String),String] = org.apache.spark.graphx.impl.GraphImpl@1ed9e2ce


In [7]:
// Let's look at the nodes
graph.vertices.foreach(println)

(7,(jgonzal,postdoc))
(5,(franklin,prof))
(2,(istoica,prof))
(3,(rxin,student))


In [8]:
// and the esges
graph.edges.foreach(println)

Edge(2,5,colleague)
Edge(5,7,pi)
Edge(5,3,advisor)
Edge(3,7,collab)


## Example 2: Flights

Download the data from https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/HG7NV7/EIR0RA and put it into this folder

In [9]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import scala.util.hashing.MurmurHash3
import org.apache.spark.SparkContext._

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import scala.util.hashing.MurmurHash3
import org.apache.spark.SparkContext._


In [17]:
// Read the zipped file
val df = spark
  .read
  .option("header","true")
  .csv("2008.csv.bz2")

df: org.apache.spark.sql.DataFrame = [Year: string, Month: string ... 27 more fields]


In [34]:
// What columns we have?
df.columns.foreach(println)

Year
Month
DayofMonth
DayOfWeek
DepTime
CRSDepTime
ArrTime
CRSArrTime
UniqueCarrier
FlightNum
TailNum
ActualElapsedTime
CRSElapsedTime
AirTime
ArrDelay
DepDelay
Origin
Dest
Distance
TaxiIn
TaxiOut
Cancelled
CancellationCode
Diverted
CarrierDelay
WeatherDelay
NASDelay
SecurityDelay
LateAircraftDelay


In [20]:
// Let's look at some of them
df.select(
    "FlightNum",
    "Month",
    "DayofMonth",
    "DepTime",
    "ArrTime",
    "Origin",
    "Dest").show()

+---------+-----+----------+-------+-------+------+----+
|FlightNum|Month|DayofMonth|DepTime|ArrTime|Origin|Dest|
+---------+-----+----------+-------+-------+------+----+
|      588|    1|         3|   1343|   1451|   HOU| LIT|
|     1343|    1|         3|   1125|   1247|   HOU| MAF|
|     3841|    1|         3|   2009|   2136|   HOU| MAF|
|        3|    1|         3|    903|   1203|   HOU| MCO|
|       25|    1|         3|   1423|   1726|   HOU| MCO|
|       51|    1|         3|   2024|   2325|   HOU| MCO|
|      940|    1|         3|   1753|   2053|   HOU| MCO|
|     2621|    1|         3|    622|    935|   HOU| MCO|
|      389|    1|         3|   1944|   2210|   HOU| MDW|
|      519|    1|         3|   1453|   1716|   HOU| MDW|
|      894|    1|         3|   2030|   2251|   HOU| MDW|
|      969|    1|         3|    708|    936|   HOU| MDW|
|     2174|    1|         3|   1749|   2039|   HOU| MDW|
|     2445|    1|         3|   1217|   1431|   HOU| MDW|
|     2974|    1|         3|   

In [21]:
// Creating a list of Airport COdes (for both origins and destinations)
val airportCodes = df.select($"Origin", $"Dest")
  .flatMap(x => Iterable(x(0).toString, x(1).toString))

airportCodes.show()

+-----+
|value|
+-----+
|  HOU|
|  LIT|
|  HOU|
|  MAF|
|  HOU|
|  MAF|
|  HOU|
|  MCO|
|  HOU|
|  MCO|
|  HOU|
|  MCO|
|  HOU|
|  MCO|
|  HOU|
|  MCO|
|  HOU|
|  MDW|
|  HOU|
|  MDW|
+-----+
only showing top 20 rows



airportCodes: org.apache.spark.sql.Dataset[String] = [value: string]


In [22]:
// Generate a unique Long Integer codes for each ariport
val airportVertices: RDD[(VertexId, String)] = airportCodes
  .rdd
  .distinct()
  .map(x => (MurmurHash3.stringHash(x), x))

airportVertices.toDF.show()

+-----------+---+
|         _1| _2|
+-----------+---+
|-1352140065|MSY|
|-1855207100|PSP|
| -413129232|SPS|
|  455752397|FWA|
|-2105183870|BMI|
|-1160109538|MGM|
|  891784594|SUN|
|  453299794|CAK|
|-1852287689|RIC|
|-1405933338|EWN|
|  256932131|GFK|
| 1294756036|BIS|
|-1050027927|PHF|
|-1042179113|STX|
| -170780705|FLL|
|  889896038|CHS|
|-1238667859|AGS|
|-1415751253|SRQ|
| -756929209|PVD|
| -333041992|AZO|
+-----------+---+
only showing top 20 rows



airportVertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String)] = MapPartitionsRDD[44] at map at <console>:41


In [23]:
// just in case!
val defaultAirport = ("Missing")

defaultAirport: String = Missing


In [24]:
// FInd all origin-destination pairs
val flightsFromTo = df.select($"Origin",$"Dest")

flightsFromTo.show()

+------+----+
|Origin|Dest|
+------+----+
|   HOU| LIT|
|   HOU| MAF|
|   HOU| MAF|
|   HOU| MCO|
|   HOU| MCO|
|   HOU| MCO|
|   HOU| MCO|
|   HOU| MCO|
|   HOU| MDW|
|   HOU| MDW|
|   HOU| MDW|
|   HOU| MDW|
|   HOU| MDW|
|   HOU| MDW|
|   HOU| MDW|
|   HOU| MSY|
|   HOU| MSY|
|   HOU| MSY|
|   HOU| MSY|
|   HOU| MSY|
+------+----+
only showing top 20 rows



flightsFromTo: org.apache.spark.sql.DataFrame = [Origin: string, Dest: string]


In [25]:
// Now count the pairs
val flightEdges = flightsFromTo.map(x => ( (
        MurmurHash3.stringHash(x(0).toString),    // x._1._1 or 1st part of the key
        MurmurHash3.stringHash(x(1).toString)),   // x._1._2 or 2nd part of the key
        1)                                        // x._2    or value
    )
    .rdd
    .reduceByKey(_+_)
    .map(x => Edge(x._1._1, x._1._2, x._2))

flightEdges.toDF.show()

+-----------+-----------+----+
|      srcId|      dstId|attr|
+-----------+-----------+----+
| 1567376521| 1710100676| 297|
| 1200735862|  384808397|   1|
| 1917476200|  813811672|1183|
|-1928985797| -216730884| 509|
| -756929209|-1182139023| 840|
| 1226329766| 1554094361| 181|
|  -44023576|-1011165063| 121|
|-1295407781|-1082293340| 676|
|  178004914| 1838031709| 138|
|-1753379389|-1011165063|  66|
| -297351069|-1679326034| 896|
| -520037749|-1745667420| 586|
| 1014929201|-1352361520| 837|
|-1358725746| -520037749|  14|
| 1228942540| 1169375524| 143|
|-1810763542|-1736145510|  60|
|  265524642|-1181519057|   1|
| 1710100676| 1936766418| 889|
| 1917476200|  -57532706| 359|
|-1859193672|-1728615855|   1|
+-----------+-----------+----+
only showing top 20 rows



flightEdges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[60] at map at <console>:44


In [26]:
// Build a graph
val graph = Graph(airportVertices, flightEdges, defaultAirport)

graph: org.apache.spark.graphx.Graph[String,Int] = org.apache.spark.graphx.impl.GraphImpl@61475f3c


In [27]:
// Take a quick look at the edges
graph.edges.take(10).foreach(println)

Edge(-2119762115,-193417555,1837)
Edge(-2119762115,-110772504,937)
Edge(-2105183870,-1737322424,50)
Edge(-2105158920,-1954981156,291)
Edge(-2105158920,-1753379389,1)
Edge(-2105158920,-1448961741,191)
Edge(-2105158920,-520037749,793)
Edge(-2029236616,1710100676,682)
Edge(-2017080169,-1737322424,121)
Edge(-2017080169,-1227538930,121)


In [28]:
// What is the code of LAX?
graph.vertices.filter{ case (vid, airportCodes) => airportCodes == "LAX" }.take(10) //-44023576

res15: Array[(org.apache.spark.graphx.VertexId, String)] = Array((-44023576,LAX))


In [29]:
// What are some of the origin airports to LAX?
graph.edges.filter{e => e.srcId == -44023576 }.take(10).foreach(println)

Edge(-44023576,-1885570066,127)
Edge(-44023576,-1737322424,4142)
Edge(-44023576,-1345915352,202)
Edge(-44023576,-1339998817,445)
Edge(-44023576,-1288021106,576)
Edge(-44023576,-1227538930,816)
Edge(-44023576,-1182139023,740)
Edge(-44023576,-1069680510,1)
Edge(-44023576,-1020692585,2831)
Edge(-44023576,-1011165063,121)


In [30]:
// How many flights from LAX?
graph.edges.filter{e => e.srcId == -44023576 }.count

res17: Long = 86


In [31]:
//How many flights for each pair of origin-destination ariports?
graph.triplets.map(triplet => triplet.attr
  + "\tflights from "
  + triplet.srcAttr 
  + " to " 
  + triplet.dstAttr).collect.foreach(println(_))

1837	flights from SNA to PHX
937	flights from SNA to SEA
50	flights from BMI to LAS
291	flights from ATW to CVG
1	flights from ATW to DSM
191	flights from ATW to MKE
793	flights from ATW to ORD
682	flights from TLH to ATL
121	flights from AMA to LAS
121	flights from AMA to ABQ
899	flights from AMA to DAL
253	flights from AMA to IAH
456	flights from ERI to DTW
203	flights from COS to MSP
445	flights from COS to PHX
234	flights from COS to SAN
347	flights from COS to LAX
1	flights from COS to FAT
105	flights from CVG to IND
265	flights from CVG to RSW
310	flights from CVG to AUS
402	flights from CVG to BWI
387	flights from CVG to TPA
322	flights from CVG to CMH
474	flights from CVG to JFK
327	flights from CVG to OKC
81	flights from CVG to AVP
528	flights from CVG to DEN
327	flights from CVG to DAY
10	flights from CVG to AZO
335	flights from CVG to FLL
1114	flights from CVG to DTW
21	flights from CVG to EVV
1	flights from CVG to SBN
127	flights from CVG to BGR
484	flights from CVG to MIA


121	flights from DEN to HNL
740	flights from DEN to EGE
592	flights from DEN to BOI
95	flights from DEN to BDL
274	flights from DEN to SBA
117	flights from DEN to RDU
1	flights from DEN to BTR
156	flights from DEN to GRR
436	flights from DEN to FAT
584	flights from DEN to MEM
198	flights from DEN to SDF
978	flights from DEN to IAD
1072	flights from DEN to LGA
768	flights from DEN to TUS
153	flights from DEN to BFL
1	flights from DEN to HLN
242	flights from DEN to FCA
78	flights from DEN to ANC
560	flights from DEN to CPR
703	flights from DEN to OMA
51	flights from MLI to LAS
142	flights from MLI to DFW
591	flights from DCA to CVG
66	flights from DCA to DSM
430	flights from DCA to STL
11	flights from DCA to SRQ
527	flights from DCA to TPA
225	flights from DCA to PVD
988	flights from DCA to IAH
600	flights from DCA to MCO
77	flights from DCA to MSN
986	flights from DCA to DTW
104	flights from DCA to MHT
922	flights from DCA to RDU
320	flights from DCA to CLE
3865	flights from DCA to LGA


800	flights from LAX to MDW
118	flights from LAX to EUG
182	flights from LAX to ASE
345	flights from LAX to ELP
8	flights from LAX to GJT
70	flights from LAX to BDL
1562	flights from LAX to SBA
253	flights from LAX to CLE
1421	flights from LAX to FAT
242	flights from LAX to MEM
3302	flights from LAX to SJC
329	flights from LAX to IYK
469	flights from LAX to HOU
1811	flights from LAX to ATL
44	flights from LAX to SJU
5	flights from LAX to ANC
310	flights from PNS to CLT
168	flights from PNS to ORD
617	flights from PNS to DFW
533	flights from MOD to SFO
248	flights from ROA to DTW
181	flights from LIH to OGG
17	flights from LIH to DEN
107	flights from LIH to PHX
121	flights from LIH to SEA
129	flights from TUL to LAS
483	flights from TUL to DEN
729	flights from TUL to ORD
341	flights from TUL to PHX
114	flights from TUL to SMF
1	flights from TUL to ASE
2	flights from TUL to GJT
433	flights from TUL to HOU
642	flights from TUL to ATL
336	flights from TUL to MCI
462	flights from DHN to ATL

1158	flights from GRR to ORD
840	flights from GRR to DTW
242	flights from GRR to DFW
413	flights from GRR to CLE
2	flights from TWF to SUN
227	flights from PBI to CVG
62	flights from PBI to LAS
442	flights from PBI to TPA
714	flights from PBI to CLT
720	flights from PBI to PHL
1168	flights from PBI to JFK
95	flights from PBI to DEN
382	flights from PBI to DCA
212	flights from PBI to DTW
323	flights from PBI to DFW
371	flights from PBI to ISP
374	flights from PBI to HPN
435	flights from FAT to DEN
215	flights from FAT to LGB
571	flights from FAT to PHX
256	flights from FAT to SAN
363	flights from MEM to TLH
519	flights from MEM to CVG
97	flights from MEM to SAV
340	flights from MEM to DSM
43	flights from MEM to RSW
363	flights from MEM to STL
330	flights from MEM to MKE
324	flights from MEM to TPA
361	flights from MEM to HSV
585	flights from MEM to DEN
339	flights from MEM to SAT
23	flights from MEM to LEX
121	flights from MEM to MSN
363	flights from MEM to LAX
363	flights from MEM to P

127	flights from EWR to AVL
222	flights from EWR to MDW
1398	flights from EWR to DTW
461	flights from EWR to MHT
121	flights from EWR to HNL
1183	flights from EWR to DFW
435	flights from EWR to BDL
320	flights from EWR to SDF
682	flights from EWR to IAD
309	flights from EWR to SLC
454	flights from EWR to ROC
478	flights from EWR to BTV
120	flights from JAN to CVG
227	flights from JAN to BWI
104	flights from JAN to ORD
578	flights from JAN to IAH
227	flights from JAN to MDW
893	flights from JAN to ATL
178	flights from MCI to COS
512	flights from MCI to CVG
432	flights from MCI to IND
5	flights from MCI to RSW
707	flights from MCI to LAS
2	flights from MCI to MKE
236	flights from MCI to MSY
1684	flights from MCI to DAL
433	flights from MCI to OKC
199	flights from MCI to PIT
437	flights from MCI to BNA
769	flights from MCI to IAH
480	flights from MCI to MCO
121	flights from MCI to SEA
452	flights from MCI to LAX
489	flights from MCI to DTW
34	flights from MCI to MIA
421	flights from MCI t

...

684	flights from ATL to FAY
299	flights from ATL to CHO
541	flights from ATL to PVD
859	flights from ATL to GSO
825	flights from ATL to SAT
1946	flights from ATL to JAX
597	flights from ATL to SEA
119	flights from ATL to CID
1814	flights from ATL to LAX
1402	flights from ATL to PNS
90	flights from ATL to OAK
465	flights from ATL to AVL
578	flights from ATL to XNA
214	flights from ATL to SMF
692	flights from ATL to MLB
347	flights from ATL to MHT
110	flights from ATL to FSD
429	flights from ATL to EVV
625	flights from ATL to ILM
1263	flights from ATL to CAK
345	flights from ATL to FWA
339	flights from ATL to VLD
3194	flights from ATL to DFW
944	flights from ATL to BTR
449	flights from ATL to GRR
216	flights from ATL to SJC
1732	flights from ATL to IAD
486	flights from ATL to MYR
882	flights from ATL to ROC
333	flights from ATL to MLU
229	flights from ATL to TUS
93	flights from ATL to GRB
1191	flights from ATL to HOU
111	flights from ATL to BTV
119	flights from ATL to TOL
1074	flights fr

242	flights from SAV to DFW
1	flights from SAV to CHS
396	flights from SAV to IAD
268	flights from STT to SJU
72	flights from DSM to LAX
340	flights from DSM to DTW
698	flights from DSM to DFW
318	flights from DSM to MEM
60	flights from RNO to SNA
117	flights from RNO to GEG
236	flights from RNO to LGB
967	flights from RNO to PHX
346	flights from RNO to SEA
436	flights from RNO to OAK
126	flights from RNO to ONT
1	flights from BHM to MKE
1	flights from BHM to AUS
339	flights from BHM to BWI
344	flights from BHM to MSY
200	flights from BHM to CLT
121	flights from BHM to JAX
233	flights from BHM to MDW
42	flights from BHM to CLE
235	flights from BHM to HOU
378	flights from RSW to MSP
202	flights from RSW to BUF
687	flights from RSW to CLT
85	flights from RSW to CMH
882	flights from RSW to JFK
120	flights from RSW to BDL
1495	flights from RSW to ATL
93	flights from RSW to FNT
235	flights from RSW to HPN
51	flights from LAS to BMI
50	flights from LAS to COS
981	flights from LAS to MSP
526	

374	flights from PHL to MSY
322	flights from PHL to CMH
1019	flights from PHL to PIT
45	flights from PHL to SAT
121	flights from PHL to ALB
2360	flights from PHL to MCO
531	flights from PHL to JAX
155	flights from PHL to SEA
736	flights from PHL to LAX
885	flights from PHL to DTW
1142	flights from PHL to DFW
1100	flights from PHL to RDU
719	flights from PHL to PBI
346	flights from PHL to MEM
233	flights from PHL to SLC
3	flights from PHL to LGA
567	flights from PHL to SJU
123	flights from SGF to CVG
1072	flights from SGF to DFW
362	flights from SGF to MEM
121	flights from MGM to IAH
363	flights from AEX to DFW
386	flights from CAE to CVG
117	flights from CAE to IAH
242	flights from CAE to DTW
228	flights from CAE to MEM
479	flights from CAE to IAD
205	flights from CAE to LGA
111	flights from HDN to DFW
97	flights from HDN to ATL
903	flights from DAL to STL
2304	flights from DAL to AUS
696	flights from DAL to MSY
666	flights from DAL to MAF
3260	flights from DAL to HOU
1667	flights from

14	flights from LAX to MTJ
2653	flights from LAX to JFK
2415	flights from LAX to ORD
543	flights from LAX to SAT
1228	flights from LAX to IAH
2168	flights from LAX to SEA
445	flights from LAX to MOD
2750	flights from LAX to OAK
341	flights from LAX to YUM
157	flights from LAX to ICT
93	flights from LAX to EGE
341	flights from LAX to BOI
7	flights from LAX to RDU
1742	flights from LAX to SLC
1450	flights from LAX to TUS
242	flights from LAX to BFL
1059	flights from LAX to EWR
460	flights from LAX to MCI
363	flights from PNS to MEM
1	flights from MOD to ONT
1	flights from MOD to BFL
1	flights from ROA to CLT
200	flights from ROA to IAD
478	flights from ROA to ATL
23	flights from LIH to SNA
144	flights from LIH to SFO
270	flights from LIH to LAX
225	flights from TUL to STL
118	flights from TUL to ABQ
763	flights from TUL to DAL
88	flights from TUL to SAT
8	flights from TUL to MCO
117	flights from TUL to SAN
1174	flights from TUL to DFW
363	flights from TUL to MEM
117	flights from TUL to O

2595	flights from ATL to FLL
1850	flights from ATL to BOS
954	flights from ATL to SAN
479	flights from ATL to ROA
462	flights from ATL to DHN
634	flights from ATL to MDT
5	flights from ATL to ASE
354	flights from ATL to SBN
97	flights from ATL to EGE
592	flights from ATL to TRI
222	flights from ATL to GRK
1123	flights from ATL to GPT
1431	flights from ATL to CHS
966	flights from ATL to CLE
1647	flights from ATL to MEM
871	flights from ATL to SDF
1038	flights from ATL to SLC
321	flights from ATL to PWM
3676	flights from ATL to LGA
272	flights from ATL to MCN
953	flights from ATL to VPS
332	flights from ATL to ONT
214	flights from ATL to ISP
2502	flights from ATL to EWR
1271	flights from ATL to MCI
42	flights from ATL to JAC
555	flights from ATL to OMA
153	flights from BFL to DEN
117	flights from BFL to SMF
1	flights from BFL to FAT
84	flights from SUX to MSP
66	flights from FNT to LAS
1074	flights from FNT to ATL
121	flights from HLN to MSP
121	flights from SJU to STT
314	flights from S